<a href="https://colab.research.google.com/github/ArunVignesh75/Machine-Learning/blob/main/dhvani_CNN_Assessment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Image Classification using CNN Model

#contents:-

1. Dataset Preparation

2. Data Preprocessing

3. Annotation Conversions

4. Iitialising the model

5. Model Predictions

6. Evaluating the Model

In [ ]:
!pip install -q kaggle

In [ ]:
!pip install xmltodict


In [ ]:
import os
import xmltodict
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split

In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"arunvigneshp","key":"9b17a7c8bc86f8b2fb5684357a8f6d63"}'}

In [ ]:
! mkdir ~/.kaggle

In [ ]:
! cp kaggle.json ~/.kaggle/

In [ ]:
! chmod 600 ~/.kaggle.json

chmod: cannot access '/root/.kaggle.json': No such file or directory


In [ ]:
! kaggle datasets list

ref                                                      title                                           size  lastUpdated          downloadCount  voteCount  usabilityRating  
-------------------------------------------------------  ---------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
nelgiriyewithana/top-spotify-songs-2023                  Most Streamed Spotify Songs 2023                47KB  2023-08-26 11:04:57           4744        150  1.0              
nelgiriyewithana/global-youtube-statistics-2023          Global YouTube Statistics 2023                  60KB  2023-07-28 15:36:38          15801        529  1.0              
joebeachcapital/students-performance                     Students Performance                             2KB  2023-08-31 00:50:11           1250         33  1.0              
inductiveanks/top-1000-imdb-movies-dataset               Top 1000 IMDb Movies Dataset                    91KB  2023-08-0

In [ ]:
!kaggle datasets download -d pratikbarua/vehicle-detection-dataset

100% 1.39G/1.39G [00:08<00:00, 126MB/s] 
100% 1.39G/1.39G [00:08<00:00, 176MB/s]


In [ ]:
!unzip vehicle-detection-dataset.zip

Streaming output truncated to the last 5000 lines.
  inflating: train/Final Train Dataset/Dipto_ 311.jpg  
  inflating: train/Final Train Dataset/Dipto_ 311.xml  
  inflating: train/Final Train Dataset/Dipto_ 312.jpg  
  inflating: train/Final Train Dataset/Dipto_ 312.xml  
  inflating: train/Final Train Dataset/Dipto_ 313.jpg  
  inflating: train/Final Train Dataset/Dipto_ 313.xml  
  inflating: train/Final Train Dataset/Dipto_ 314.jpg  
  inflating: train/Final Train Dataset/Dipto_ 314.xml  
  inflating: train/Final Train Dataset/Dipto_ 315.jpg  
  inflating: train/Final Train Dataset/Dipto_ 315.xml  
  inflating: train/Final Train Dataset/Dipto_ 316.jpg  
  inflating: train/Final Train Dataset/Dipto_ 316.xml  
  inflating: train/Final Train Dataset/Dipto_ 317.jpg  
  inflating: train/Final Train Dataset/Dipto_ 317.xml  
  inflating: train/Final Train Dataset/Dipto_ 318.jpg  
  inflating: train/Final Train Dataset/Dipto_ 318.xml  
  inflating: train/Final Train Dataset/Dipto_ 319.jpg

In [ ]:
# Define the path to the XML annotations and YOLO output folder
xml_folder = '/content/train/Final Train Dataset'
yolo_output_folder = '/content/train/yolo_annotations'

# Create the YOLO output folder if it doesn't exist
os.makedirs(yolo_output_folder, exist_ok=True)

# Define a mapping of class names to YOLO class IDs
class_mapping = {
    'car': 0,  # Adjust class names and IDs as needed
    'bus': 1,
    'motorbike': 2,
    # Add more classes as necessary
}

# Iterate through XML files in the XML folder
for xml_file in os.listdir(xml_folder):
    if xml_file.endswith('.xml'):
        with open(os.path.join(xml_folder, xml_file), 'r') as f:
            xml_content = f.read()
            try:
                xml_dict = xmltodict.parse(xml_content)
            except Exception as e:
                print(f"Error parsing XML file {xml_file}: {e}")
                continue  # Skip this file and continue with the next one

            # Extract image dimensions (assuming all images have the same dimensions)
            width = int(xml_dict['annotation']['size']['width'])
            height = int(xml_dict['annotation']['size']['height'])

            # Check for zero width or height
            if width == 0 or height == 0:
                print(f"Warning: Zero width or height encountered for {xml_file}. Skipping...")
                continue

            # Initialize the YOLO annotation string
            yolo_annotation = ""

            # Iterate through objects in the XML file
            objects = xml_dict['annotation']['object']
            if not isinstance(objects, list):
                objects = [objects]  # Ensure objects is a list even if there's only one object

            for obj in objects:
                class_name = obj['name']
                class_id = class_mapping.get(class_name, -1)

                if class_id != -1:
                    xmin = int(obj['bndbox']['xmin'])
                    ymin = int(obj['bndbox']['ymin'])
                    xmax = int(obj['bndbox']['xmax'])
                    ymax = int(obj['bndbox']['ymax'])

                    # Calculate YOLO format coordinates
                    x_center = (xmin + xmax) / (2 * width)
                    y_center = (ymin + ymax) / (2 * height)
                    box_width = (xmax - xmin) / width
                    box_height = (ymax - ymin) / height

                    # Append the YOLO annotation line
                    yolo_annotation += f"{class_id} {x_center} {y_center} {box_width} {box_height}\n"

            # Save the YOLO annotation to a text file with the same name as the image
            image_name = os.path.splitext(xml_file)[0] + '.txt'
            with open(os.path.join(yolo_output_folder, image_name), 'w') as yolo_file:
                yolo_file.write(yolo_annotation)


Error parsing XML file 231.xml: syntax error: line 1, column 0


In [ ]:
# Define the paths to the image and annotation folders
image_folder = '/content/train/Final Train Dataset'
annotation_folder = '/content/train/yolo_annotations'

# Get the list of all image files in the image folder
image_files = os.listdir(image_folder)

# Iterate through the image files
for image_file in image_files:
    if image_file.endswith(('.jpg', '.jpeg', '.png')):
        # Extract the base filename without the extension
        base_filename = os.path.splitext(image_file)[0]

        # Check if the corresponding annotation file exists
        annotation_file = f'{base_filename}.txt'
        annotation_path = os.path.join(annotation_folder, annotation_file)

        if not os.path.exists(annotation_path):
            # If no annotation file exists, delete the image
            image_path = os.path.join(image_folder, image_file)
            os.remove(image_path)
            print(f"Deleted: {image_file}")

print("Deletion process completed.")


Deleted: 231.jpg
Deletion process completed.


In [ ]:
test_folder = '/content/test1/test'
# Number of classes
num_classes = 3  # You have 3 classes

# Define the target size for resizing the images
target_size = (224, 224)  # Adjust to your desired size

# Initialize lists to store images and annotations
images = []
annotations = []

# Iterate through the image files in the dataset folder
for image_file in os.listdir(image_folder):
    if image_file.endswith(('.jpg', '.jpeg', '.png')):  # Adjust file extensions as needed
        image_path = os.path.join(image_folder, image_file)

        # Load and preprocess the image
        image = cv2.imread(image_path)
        image = cv2.resize(image, target_size)  # Resize to the target size
        image = image / 255.0  # Normalize pixel values to the range [0, 1]

        # Append the preprocessed image to the list
        images.append(image)

        # Load the corresponding YOLO annotation file
        annotation_file = os.path.splitext(image_file)[0] + '.txt'
        annotation_path = os.path.join(annotation_folder, annotation_file)

        # Read the YOLO annotation file and parse the annotations
        with open(annotation_path, 'r') as f:
            lines = f.readlines()

        annotation = []
        for line in lines:
            parts = line.strip().split()
            class_id = int(parts[0])
            x_center, y_center, width, height = map(float, parts[1:])
            annotation.append([class_id, x_center, y_center, width, height])

        # Append the annotations to the list
        annotations.append(annotation)

# Convert the lists of images and annotations to NumPy arrays
images = np.array(images)

# Create one-hot encoded vectors for the annotations
annotations_one_hot = []
for annotation_list in annotations:
    annotation_array = np.zeros((len(annotation_list), num_classes), dtype=int)
    for i, annotation in enumerate(annotation_list):
        class_id = annotation[0]
        annotation_array[i, class_id] = 1
    annotations_one_hot.append(annotation_array)

annotations_one_hot = np.array(annotations_one_hot)



<ipython-input-15-228d68b32d0d>:55: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  annotations_one_hot = np.array(annotations_one_hot)


In [ ]:
# Concatenate the list of one-hot encoded arrays into a single NumPy array
annotations_one_hot = np.concatenate(annotations_one_hot, axis=0)

In [ ]:
# Define the input shape based on your preprocessed images
input_shape = (224, 224, 3)  # Adjust based on your image size and channels



# Create a Sequential model
model = Sequential()

# Convolutional layers
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

# Flatten the feature maps
model.add(Flatten())

# Fully connected layers
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))  # Optional dropout layer for regularization
model.add(Dense(num_classes, activation='softmax'))  # Updated for 3 classes

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Display the model summary
model.summary()



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 222, 222, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 111, 111, 32)      0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 109, 109, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 54, 54, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 52, 52, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 26, 26, 128)       0

In [ ]:
# Train the model on the entire preprocessed dataset
batch_size = 32
epochs = 10  # Adjust as needed
# Train the model
history = model.fit(
    images, annotations_one_hot,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=0.2  # You can specify a validation split here
)

Epoch 1/10
72/72 [==============================] - 234s 3s/step - loss: 1.1856 - accuracy: 0.4748 - val_loss: 1.0221 - val_accuracy: 0.5105
Epoch 2/10
72/72 [==============================] - 234s 3s/step - loss: 1.0447 - accuracy: 0.4827 - val_loss: 1.0428 - val_accuracy: 0.5105
Epoch 3/10
72/72 [==============================] - 218s 3s/step - loss: 1.0432 - accuracy: 0.4862 - val_loss: 1.0257 - val_accuracy: 0.5105
Epoch 4/10
72/72 [==============================] - 230s 3s/step - loss: 1.0394 - accuracy: 0.4858 - val_loss: 1.0217 - val_accuracy: 0.5105
Epoch 5/10
72/72 [==============================] - 230s 3s/step - loss: 1.0340 - accuracy: 0.4858 - val_loss: 1.0391 - val_accuracy: 0.5105
Epoch 6/10
72/72 [==============================] - 226s 3s/step - loss: 1.0268 - accuracy: 0.4919 - val_loss: 1.0335 - val_accuracy: 0.5087
Epoch 7/10
72/72 [==============================] - 224s 3s/step - loss: 0.9845 - accuracy: 0.5042 - val_loss: 1.0739 - val_accuracy: 0.4808
Epoch 8/10
72

In [ ]:
# Load and preprocess the test dataset
test_images = []
test_image_paths = os.listdir(test_folder)
for test_image_path in test_image_paths:
    if test_image_path.endswith(('.jpg', '.jpeg', '.png')):
        test_image_path = os.path.join(test_folder, test_image_path)
        test_image = cv2.imread(test_image_path)
        test_image = cv2.resize(test_image, target_size)
        test_image = test_image / 255.0
        test_images.append(test_image)

# Convert the test images to a NumPy array
test_images = np.array(test_images)

# Make predictions on the test dataset
test_predictions = model.predict(test_images)

# You can work with 'test_predictions' to analyze the model's predictions on the test data


16/16 [==============================] - 12s 767ms/step


In [ ]:
test_predictions

array([[4.0602019e-01, 4.8035479e-01, 1.1362500e-01],
       [5.5296099e-01, 3.0227005e-01, 1.4476898e-01],
       [5.7445681e-01, 1.0917171e-01, 3.1637153e-01],
       ...,
       [4.7595167e-01, 2.1573621e-01, 3.0831215e-01],
       [2.8007606e-01, 2.8407305e-01, 4.3585086e-01],
       [9.2969310e-01, 7.0218511e-02, 8.8362278e-05]], dtype=float32)

In [ ]:
# Define a dictionary to map class indices to class names
class_names = {
    0: 'car',
    1: 'bus',
    2: 'motorbike'
}

# Convert the predictions to class indices (0, 1, or 2)
predicted_classes = np.argmax(test_predictions, axis=1)

# Map the class indices to class names
predicted_class_names = [class_names[class_idx] for class_idx in predicted_classes]

# Display the predicted class names for each test image
for i, image_path in enumerate(test_image_paths):
    if image_path.endswith(('.jpg', '.jpeg', '.png')):
        print(f"Image: {image_path}, Predicted Class: {predicted_class_names[i]}")

Image: Shykat_4_ (61)_jpg.rf.0b5df937ad66e8f79809e548ee2953af.jpg, Predicted Class: bus
Image: Shykat_4_ (23)_jpg.rf.c29fba7c4c2b0d2ee12795587ded84e8.jpg, Predicted Class: car
Image: Shykat_5_ (44)_jpg.rf.79fa6df6074d69e1e09e78447dc67190.jpg, Predicted Class: car
Image: Shykat_02_006_jpg.rf.cbaffebd7c9b1a7917969838fe4ec596.jpg, Predicted Class: car
Image: Shykat_03_025_jpg.rf.236f3bd3394f66974e29eb03dc89904e.jpg, Predicted Class: motorbike
Image: Shykat_03_028_jpg.rf.5e8d585a92a192ca9b045ca8d949ba79.jpg, Predicted Class: car
Image: Shykat_5_ (113)_jpg.rf.1d22d7b62dd7c899ecc544a7ac6340be.jpg, Predicted Class: bus
Image: Shykat_5_ (99)_jpg.rf.77dadbce8b05681b3f945041c4eb4a74.jpg, Predicted Class: motorbike
Image: sabiha(300)_jpg.rf.5c95cec0762afb2c66e8e438911788b4.jpg, Predicted Class: car
Image: sabiha(262)_jpg.rf.5d3ccab3f3a933c9e7e3e57299234bc6.jpg, Predicted Class: car
Image: Shykat_5_ (79)_jpg.rf.d2a2fe8d33717021f2a2dd0803cba759.jpg, Predicted Class: car
Image: Shykat_02_022_jpg.rf.